## MongoDB 문제 풀어보기

[notion link](https://www.notion.so/MongoDB-845d3b14f8c84ace9044f4fd783ace7c?pvs=4)

- **복습할 개념 체크 리스트**
  - Python 함수
  - `find`
  - `sort`
  - `update_many`
  - `limit`
  - `aggregate`


## import


In [13]:
from pymongo import MongoClient
from faker import Faker
import random

## 함수 정의부


In [41]:
def generate_sample_book():
    return {
        "title": fake.sentence(nb_words=3),
        "author": fake.word(
            ext_word_list=["봉준호", "James Cameron", "Christopher Nolan", "Robert Arnold"]
        ),
        "year": random.randint(1900, 2024),
        "genre": fake.word(ext_word_list=["comic", "fantasy", "thriller", "horror"]),
    }


def generate_sample_movie():
    return {
        "title": fake.sentence(nb_words=3),
        "director": fake.word(
            ext_word_list=["봉준호", "James Cameron", "Christopher Nolan", "Robert Arnold"]
        ),
        "year": random.randint(1900, 2024),
        "rating": random.uniform(1, 10),
        "genre": fake.word(ext_word_list=["comic", "fantasy", "thriller", "horror"]),
    }


def generate_user_action():
    return {
        "user_id": random.randint(1, 10),
        "action": random.choice(ACTIONS),
        "timestamp": fake.date_time_between(),
    }


def insert_data(db):
    # 책 데이터 삽입
    db["books"].insert_many([generate_sample_book() for _ in range(50)])

    # 영화 데이터 삽입
    db["movies"].insert_many([generate_sample_movie() for _ in range(50)])

    # 사용자 행동 데이터 삽입
    db["user_actions"].insert_many([generate_user_action() for _ in range(50)])

    print("Data inserted successfully")


def find_books_by_genre(db, genre):
    books_collection = db.books
    query = {"genre": genre}
    projection = {
        "_id": 0,
        "title": 1,
        "author": 1,
        "genre": 1,
    }  # 1 은 포함할 필드, 0 은 제외할 필드

    books = books_collection.find(query, projection)
    for book in books:
        print(book)


def find_recent_actions_by_user(db, user_id, limit=5):
    """
    사용자의 최근 행동 5개를 시간 순으로 정렬하여 보여주세요
    """
    _match = {"$match": {"user_id": user_id}}
    _limit = {"$limit": limit}
    pipeline = [_match, _limit]
    result = db["user_actions"].aggregate(pipeline)

    for action in result:
        print(action)


def count_books_by_year(db):
    pass


def update_user_actions_before_date(db, user_id, date, old_action, new_action):
    pass

## db 객체 초기화 및 실습 진행


In [37]:
ACTIONS = ["click", "view", "purchase"]
fake = Faker()

client = MongoClient("mongodb://localhost:27017/")
db = client.test_database

## 데이터 삽입코드


In [34]:
insert_data(db)

Data inserted successfully


## [문제 1: 특정 장르의 책 찾기]

- **문제 설명**:
  사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
- **쿼리 작성 목표**:
  "fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.


In [38]:
find_books_by_genre(db, "fantasy")

{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami', 'genre': 'fantasy'}
{'title': '1Q84', 'author': 'Haruki Murakami', 'genre': 'fantasy'}
{'title': 'Affect fly.', 'author': 'Robert Arnold', 'genre': 'fantasy'}
{'title': 'Line raise gun where.', 'author': 'Robert Arnold', 'genre': 'fantasy'}
{'title': 'Fight join push.', 'author': 'Christopher Nolan', 'genre': 'fantasy'}
{'title': 'Put group information.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'Leg remember than middle.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'Weight seem money.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'Eat you.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'National.', 'author': '봉준호', 'genre': 'fantasy'}
{'title': 'Rise discussion.', 'author': 'Christopher Nolan', 'genre': 'fantasy'}
{'title': 'Always source from.', 'author': '봉준호', 'genre': 'fantasy'}
{'title': 'Official central doctor.', 'author': 'Robert Arnold', 'genre': 'fantasy'}
{'ti

## [문제 2: 감독별 평균 영화 평점 계산]

- **문제 설명**:
  각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
- **쿼리 작성 목표**:
  모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[Aggregation Pipelines](https://www.mongodb.com/docs/manual/core/aggregation-pipeline/#std-label-aggregation-pipeline)**

- aggregation pipeline은 하나 이상의 stage로 구성되어 있습니다:
- 각 stage당 필터, 그룹, 집계 연산 중 하나를 수행합니다.
- aggregation pipeline은 도큐먼트의 그룹을 리턴합니다. 예를 들어, 총합, 평균, 최대, 최소를 구할 수 있습니다.


In [39]:
group = {"$group": {"_id": "$director", "average_rating": {"$avg": "$rating"}}}
sort = {"$sort": {"average_rating": -1}}

pipeline = [group, sort]

for e in db.movies.aggregate(pipeline):
    print(e)

{'_id': 'Nicole Contreras', 'average_rating': 9.985575640949612}
{'_id': 'Ryan Garcia', 'average_rating': 9.952724359278863}
{'_id': 'Emily Lamb', 'average_rating': 9.938398814202332}
{'_id': 'James Phillips', 'average_rating': 9.879404744672382}
{'_id': 'Mrs. Sydney Collins', 'average_rating': 9.710170734941597}
{'_id': 'Carla Richards', 'average_rating': 9.705999993713801}
{'_id': 'Chelsea Pearson', 'average_rating': 9.650697466320608}
{'_id': 'Miss Amy Bartlett', 'average_rating': 9.43596882765703}
{'_id': 'Frances Davidson', 'average_rating': 9.387763550926685}
{'_id': 'Michael Stewart', 'average_rating': 9.319807271237838}
{'_id': 'Kathleen Moore', 'average_rating': 9.297501494033025}
{'_id': 'Steven Garrison', 'average_rating': 9.204465366232206}
{'_id': 'Briana Lee', 'average_rating': 9.177891096704212}
{'_id': 'Joseph Long', 'average_rating': 9.070833768046004}
{'_id': 'Richard Parks', 'average_rating': 8.977464471631112}
{'_id': 'Deborah Cunningham', 'average_rating': 8.971759

## [문제 3: 특정 사용자의 최근 행동 조회]

- **문제 설명**:
  특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
- **쿼리 작성 목표**:
  사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[match](https://www.mongodb.com/docs/manual/reference/operator/aggregation/match/#mongodb-pipeline-pipe.-match)**

aggregate의 탐색범위를 줄이는 데 사용됩니다.

예시코드:

```javascript
db.articles.aggregate([{ $match: { author: "dave" } }]);
```


In [42]:
find_recent_actions_by_user(db, 1, 5)

{'_id': ObjectId('662a1eb0ee11c3251e5e953e'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}
{'_id': ObjectId('662a1eb0ee11c3251e5e953f'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662a20dddecdb0e59fd25188'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}
{'_id': ObjectId('662a20dddecdb0e59fd25189'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('66305bea38803bc54e429b64'), 'user_id': 1, 'action': 'purchase', 'timestamp': datetime.datetime(2023, 8, 7, 14, 31, 15, 466000)}


## [문제 4: 출판 연도별 책의 수 계산]

- **문제 설명** :
  데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
  각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.


## [문제 5: 특정 사용자의 행동 유형 업데이트]

- **문제 설명**:
  특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
- **쿼리 작성 목표**:
  사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.


Data inserted successfully


In [ ]:
find_books_by_genre(db, "fantasy")